In [1]:
import pandas as pd
from pyspark import SparkContext
from pyspark.sql import SQLContext, SparkSession
# sc = SparkContext() 
# sqlContext = SQLContext(sc)
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.clustering import LDA, LDAModel
from pyspark.ml.feature import CountVectorizer , IDF
from nltk.stem.wordnet import WordNetLemmatizer
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, Word2Vec
from nltk.corpus import stopwords
import re as re



In [2]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [3]:
spark = SparkSession\
    .builder\
    .master("local[*]")\
    .appName("SpotifyPodcastClassification")\
    .config('spark.driver.memory', '12g')\
    .getOrCreate()


21/12/02 17:10:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / (np.sqrt(np.dot(v2, v2))+.1)


In [37]:
df = spark.read.options(inferSchema='True',delimiter=',',header='True') \
    .csv("../script_output/episode_transcript_data_w_metadata.csv")

# df = df.withColumnRenamed('_c0', 'index')
df.show()

21/12/02 17:25:58 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_uri, show_name, show_description, publisher, language, rss_link, episode_uri, episode_name, episode_description, duration, show_filename_prefix, episode_filename_prefix, Unnamed: 0, episode, transcript
 Schema: _c0, show_uri, show_name, show_description, publisher, language, rss_link, episode_uri, episode_name, episode_description, duration, show_filename_prefix, episode_filename_prefix, Unnamed: 0, episode, transcript
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv


+---+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------------------+----------+--------------------+--------------------+
|_c0|            show_uri|           show_name|    show_description|           publisher| language|            rss_link|         episode_uri|        episode_name| episode_description|          duration|show_filename_prefix|episode_filename_prefix|Unnamed: 0|             episode|          transcript|
+---+--------------------+--------------------+--------------------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+------------------+--------------------+-----------------------+----------+--------------------+--------------------+
|  0|spotify:show:2NYt...|Kream in your Koffee|A 20-something bl...|         Katie Houle|   ['en'

In [38]:
df_clean = df.select('_c0', 'episode_uri', 'show_description')
df_clean.show()

+---+--------------------+--------------------+
|_c0|         episode_uri|    show_description|
+---+--------------------+--------------------+
|  0|spotify:episode:0...|A 20-something bl...|
|  1|spotify:episode:0...|Ever wonder what ...|
|  2|spotify:episode:0...|Inside the 18 is ...|
|  3|spotify:episode:0...|Your favorite pod...|
|  4|spotify:episode:0...|The comedy podcas...|
|  5|spotify:episode:0...|Podcasts useful f...|
|  6|spotify:episode:0...|Enter the world o...|
|  7|spotify:episode:0...|We are four, 30 s...|
|  8|spotify:episode:0...|Get ready to whit...|
|  9|spotify:episode:0...|Let’s be real and...|
| 10|spotify:episode:0...|Behaviorbabe (aka...|
| 11|spotify:episode:0...|In dance, the poc...|
| 12|spotify:episode:0...|Welcome to the of...|
| 13|spotify:episode:0...|Let other student...|
| 14|spotify:episode:0...|Welcome to The Po...|
| 15|spotify:episode:0...|The eCommerce Lif...|
| 16|spotify:episode:0...|Because there’s a...|
| 17|spotify:episode:0...|Does music mak

21/12/02 17:25:59 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv


In [39]:
transcript = df_clean.rdd.map(lambda x : x['show_description']).filter(lambda x: x is not None)
StopWords = stopwords.words("english")
lmtzr = WordNetLemmatizer()

In [40]:
tokens = transcript.map( lambda document: document.strip().lower())               \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \
    .map( lambda word: [x for x in word if x not in ['welcome', 'episode', 'hello']])    \
    .map( lambda word: [lmtzr.lemmatize(x,'v') for x in word]) \
    .zipWithIndex()

21/12/02 17:26:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv


In [41]:
df_txts = spark.createDataFrame(tokens, ["list_of_words",'index'])
df_txts.show()

21/12/02 17:26:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv


+--------------------+-----+
|       list_of_words|index|
+--------------------+-----+
|[blunt, female, t...|    0|
|[ever, wonder, mu...|    1|
|[inside, source, ...|    2|
|[favorite, podcas...|    3|
|[comedy, podcast,...|    4|
|[podcast, useful,...|    5|
|[enter, world, do...|    6|
|[somethings, live...|    7|
|[ready, whiten, k...|    8|
|[real, life, chal...|    9|
|[behaviorbabe, am...|   10|
|[pocket, place, m...|   11|
|[official, will, ...|   12|
|[students, help, ...|   13|
|[pole, dance, tal...|   14|
|[ecommerce, lifes...|   15|
|[million, ways, c...|   16|
|[music, make, fee...|   17|
|[much, enough, ta...|   18|
|[take, powerful, ...|   19|
+--------------------+-----+
only showing top 20 rows



21/12/02 17:26:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv
Traceback (most recent call last):
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zi

In [42]:
stopwrmv = StopWordsRemover(inputCol = 'list_of_words', outputCol = 'tokens_sw_removed')

df_txts = stopwrmv.transform(df_txts)
df_txts.show(3)

+--------------------+-----+--------------------+
|       list_of_words|index|   tokens_sw_removed|
+--------------------+-----+--------------------+
|[blunt, female, t...|    0|[blunt, female, t...|
|[ever, wonder, mu...|    1|[ever, wonder, mu...|
|[inside, source, ...|    2|[inside, source, ...|
+--------------------+-----+--------------------+
only showing top 3 rows



21/12/02 17:26:20 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv
Traceback (most recent call last):
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zi

In [43]:
# TF
cv = CountVectorizer(inputCol="list_of_words", outputCol="raw_features", vocabSize=5000, minDF=10.0)
cvmodel = cv.fit(df_txts)
result_cv = cvmodel.transform(df_txts)
result_cv.show()

21/12/02 17:26:21 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv
21/12/02 17:26:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv


+--------------------+-----+--------------------+--------------------+
|       list_of_words|index|   tokens_sw_removed|        raw_features|
+--------------------+-----+--------------------+--------------------+
|[blunt, female, t...|    0|[blunt, female, t...|(5000,[20,26,31,6...|
|[ever, wonder, mu...|    1|[ever, wonder, mu...|(5000,[1,20,22,66...|
|[inside, source, ...|    2|[inside, source, ...|(5000,[6,9,14,23,...|
|[favorite, podcas...|    3|[favorite, podcas...|(5000,[0,33,52,70...|
|[comedy, podcast,...|    4|[comedy, podcast,...|(5000,[0,1,5,9,11...|
|[podcast, useful,...|    5|[podcast, useful,...|(5000,[0,88,261,8...|
|[enter, world, do...|    6|[enter, world, do...|(5000,[1,5,10,26,...|
|[somethings, live...|    7|[somethings, live...|(5000,[6,17,232,5...|
|[ready, whiten, k...|    8|[ready, whiten, k...|(5000,[1,2,10,39,...|
|[real, life, chal...|    9|[real, life, chal...|(5000,[0,7,8,9,16...|
|[behaviorbabe, am...|   10|[behaviorbabe, am...|(5000,[0,51,96,13...|
|[pock

Traceback (most recent call last):
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


In [44]:
# IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idfModel = idf.fit(result_cv)
result_tfidf = idfModel.transform(result_cv) 
result_tfidf.show()

21/12/02 17:26:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv
21/12/02 17:26:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv


+--------------------+-----+--------------------+--------------------+--------------------+
|       list_of_words|index|   tokens_sw_removed|        raw_features|            features|
+--------------------+-----+--------------------+--------------------+--------------------+
|[blunt, female, t...|    0|[blunt, female, t...|(5000,[20,26,31,6...|(5000,[20,26,31,6...|
|[ever, wonder, mu...|    1|[ever, wonder, mu...|(5000,[1,20,22,66...|(5000,[1,20,22,66...|
|[inside, source, ...|    2|[inside, source, ...|(5000,[6,9,14,23,...|(5000,[6,9,14,23,...|
|[favorite, podcas...|    3|[favorite, podcas...|(5000,[0,33,52,70...|(5000,[0,33,52,70...|
|[comedy, podcast,...|    4|[comedy, podcast,...|(5000,[0,1,5,9,11...|(5000,[0,1,5,9,11...|
|[podcast, useful,...|    5|[podcast, useful,...|(5000,[0,88,261,8...|(5000,[0,88,261,8...|
|[enter, world, do...|    6|[enter, world, do...|(5000,[1,5,10,26,...|(5000,[1,5,10,26,...|
|[somethings, live...|    7|[somethings, live...|(5000,[6,17,232,5...|(5000,[6,1

Traceback (most recent call last):
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 186, in manager
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/daemon.py", line 74, in worker
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 643, in main
    if read_int(infile) == SpecialLengths.END_OF_STREAM:
  File "/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 564, in read_int
    raise EOFError
EOFError


# Fit and Describe

In [45]:
lda = LDA(k=10, maxIter = 2)
model = lda.fit(result_tfidf.select('index', 'features'))
ll = model.logLikelihood(result_tfidf)
lp = model.logPerplexity(result_tfidf)
print("The lower bound on the log likelihood of the entire corpus: " + str(ll))
print("The upper bound on perplexity: " + str(lp))

21/12/02 17:26:42 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv
21/12/02 17:26:54 WARN DAGScheduler: Broadcasting large task binary with size 1373.2 KiB
21/12/02 17:26:54 WARN DAGScheduler: Broadcasting large task binary with size 1373.2 KiB
21/12/02 17:26:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , show_description, episode_uri
 Schema: _c0, show_description, episode_uri
Expected: _c0 but found: 
CSV file: file:///Volumes/GoogleDrive/My%20Drive/Documents/Columbia/CLASSES/!SEM%202%20-%20NLP/COLUMBIA%20SPOTIFY%20PODCAST/script_output/episode_transcript_data_w_metadata.csv
21/12/02 17:33:42 WARN CSVHeaderChecker: CSV header does not conform to the sc

The lower bound on the log likelihood of the entire corpus: -58399722.72354713
The upper bound on perplexity: 7.1869648110218005


In [46]:
# Describe topics.
topics = model.describeTopics(7)
print("The topics described by their top-weighted terms:")
topics.show()

The topics described by their top-weighted terms:
+-----+--------------------+--------------------+
|topic|         termIndices|         termWeights|
+-----+--------------------+--------------------+
|    0|[0, 16, 13, 3, 25...|[0.00611678902606...|
|    1|[10, 0, 8, 1, 17,...|[0.00685969000664...|
|    2|[2, 0, 4, 57, 5, ...|[0.00674179527515...|
|    3|[65, 260, 0, 55, ...|[0.00890301497464...|
|    4|[2, 178, 13, 8, 4...|[0.00558137219786...|
|    5|[13, 1, 61, 0, 17...|[0.00576652204050...|
|    6|[104, 3, 139, 285...|[0.00940090967704...|
|    7|[374, 7, 20, 0, 1...|[0.00855893723711...|
|    8|[145, 102, 154, 3...|[0.00858467743446...|
|    9|[82, 89, 153, 24,...|[0.01574083654863...|
+-----+--------------------+--------------------+



In [47]:
num_words = 20
num_topics = 7
# topicIndices = sc.parallelize(model.describeTopics(maxTermsPerTopic = num_words))
topic_list = topics.collect()
for i in range(0,num_topics):
    print("Topic cluster: ", i)
    for index in topic_list[i][1]:
        print(cvmodel.vocabulary[index])


Topic cluster:  0
podcast
people
best
help
asmr
support
unsolved
Topic cluster:  1
stories
podcast
make
support
live
jeremy
life
Topic cluster:  2
talk
podcast
every
media
host
stories
life
Topic cluster:  3
english
craig
podcast
teach
break
want
show
Topic cluster:  4
talk
black
best
make
every
podcast
culture
Topic cluster:  5
best
support
think
podcast
live
read
daily
Topic cluster:  6
students
help
brand
nintendo
personal
gary
support


1. Iterate through until we get meaningful results
2. Evaluation: Manually annotating - then that's our eval metric  --> give clients to think about - perhaps 
3. 

In [25]:
num_words = 20
num_topics = 7
# topicIndices = sc.parallelize(model.describeTopics(maxTermsPerTopic = num_words))
topic_list = topics.collect()
for i in range(0,num_topics):
    print("Topic cluster: ", i)
    for index in topic_list[i][1]:
        print(cvmodel.vocabulary[index])


Topic cluster:  0
happen
go
whole
things
anything
something
like
Topic cluster:  1
think
follow
everything
anything
every
life
live
Topic cluster:  2
next
good
break
care
nothing
back
would
Topic cluster:  3
many
understand
stuff
leave
would
mean
still
Topic cluster:  4
say
actually
still
person
happen
okay
hold
Topic cluster:  5
feel
make
take
say
person
best
able
Topic cluster:  6
really
work
break
actually
think
talk
whole
